<a href="https://colab.research.google.com/github/thamizhannal/eva5/blob/master/DavidNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/theschoolof-ai/JEDI.git

Cloning into 'JEDI'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 381 (delta 21), reused 9 (delta 2), pack-reused 332
Receiving objects: 100% (381/381), 241.02 MiB | 19.85 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [2]:
cd /content/JEDI/tsai.jedi

/content/JEDI/tsai.jedi


In [3]:
import sys
sys.path.append('Models/')
from S8_resnet import ResNet18
from S7 import model_summary
from S9_resnet import resnet18

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tsai.jedi.batchnorm import GhostBatchNorm

from S9_resnet import BasicBlock


#from s11 import BasicBlock
#from s11 import ResNet
from batchnorm import GhostBatchNorm

In [9]:
class DavidNet(nn.Module):
    # BN_flag 0: normal batchnorm; 1: Ghost batchnorm
    def batch_norm(self, channels, BN_flag):
        if BN_flag == 1:
            return GhostBatchNorm(channels, num_splits=2, weight=False)
        else:
            return nn.BatchNorm2d(channels)

    def __init__(self, flag):

        self.in_planes = 64

        super(DavidNet, self).__init__()
        # 32,32,3

        #prep Layer
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)  # 32,32,8
        self.Batchnorm1 = self.batch_norm(64, flag)

        # Layer1
        self.conv2 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)  # 32,32,8
        self.pool1 = nn.MaxPool2d(2, 2)
        self.Batchnorm2 = self.batch_norm(128, flag)

        self.residual1 = self._make_layer(BasicBlock, 128, num_blocks=2, stride=1)

        # Layer2
        self.conv3 = nn.Conv2d(
            in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)  # 32,32,8
        self.pool2 = nn.MaxPool2d(2, 2)
        self.Batchnorm3 = self.batch_norm(256, flag)

        # Layer3
        self.conv4 = nn.Conv2d(
            in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)  # 32,32,8
        self.pool3 = nn.MaxPool2d(2, 2)
        self.Batchnorm4 = self.batch_norm(512, flag)

        self.residual2 = self._make_layer(BasicBlock, 512, num_blocks=2, stride=1)

        self.pool4 = nn.MaxPool2d(4, 4)

        self.fc = nn.Conv2d(512, 10, kernel_size=1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # PrepLayer
        x = F.relu(self.Batchnorm1(self.conv1(x)))  # 8

        # Layer1
        x1 = F.relu(self.Batchnorm2(self.pool1(self.conv2(x))))  # 8
        x1 = torch.cat((x1, self.residual1), dim=1)

        # Layer2
        x2 = F.relu(self.Batchnorm3(self.pool2(self.conv3(x1))))

        # Layer3
        x3 = F.relu(self.Batchnorm4(self.pool3(self.conv4(x2))))  # 8
        x3 = torch.cat((x3, self.residual2), dim=1)

        #Layer4
        x4 = self.pool4(x3)

        x5 = self.fc(x4)

        x5 = x5.view(-1, 10)
        return F.log_softmax(x5, dim=-1)

In [10]:
net = DavidNet(flag=True)

In [12]:
from torchsummary import summary
summary(net, (3, 32, 32))

TypeError: ignored